In [1]:
from n2v.models import N2VConfig, N2V
import numpy as np
import tensorflow as tf

/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: 

In [2]:
X = np.random.uniform(-2,2,(10, 96, 96, 96, 1))

config = N2VConfig(X, unet_kern_size=3,
                   train_steps_per_epoch=5,train_epochs=5, train_loss='mse', batch_norm=True,
                   train_batch_size=128, n2v_perc_pix=1.6, n2v_patch_shape=(64, 64, 64),
                   n2v_manipulator='uniform_withCP', n2v_neighborhood_radius=5)

model = N2V(config, 'n2v_3D_2', basedir='models')

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
#optimizer = optimizers.SGD(lr=0.01)
#isinstance(optimizer, optimizers.Optimizer) or print(ValueError())

#tfoptimizer = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-7)
# model.prepare_for_training(optimizer = optimizers.TFOptimizer(tfoptimizer))
model.prepare_for_training()
model.keras_model._make_train_function()
model.keras_model._make_test_function()
model.keras_model._make_predict_function()
#model.prepare_for_training()
model.keras_model.summary()

Instructions for updating:
Use tf.cast instead.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, None, None, N 0                                            
__________________________________________________________________________________________________
down_level_0_no_0 (Conv3D)      (None, None, None, N 896         input[0][0]                      
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, N 128         down_level_0_no_0[0][0]          
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, N 0           batch_normalization_1[0][0]      
_____________________________________________________________

In [4]:
nodes = [n.name for n in tf.get_default_graph().as_graph_def().node]
[k for k in nodes if 'group' in k]


['training/group_deps', 'group_deps', 'group_deps_1']

In [5]:
loss = tf.get_default_graph().get_tensor_by_name("loss/mul:0")
#train_op = tfoptimizer.minimize(loss, name='train')
tf.global_variables_initializer()
tf.train.Saver().as_saver_def()

filename_tensor_name: "save/Const:0"
save_tensor_name: "save/control_dependency:0"
restore_op_name: "save/restore_all"
max_to_keep: 5
keep_checkpoint_every_n_hours: 10000.0
version: V2

In [6]:
 tf.summary.FileWriter("logs", tf.get_default_graph())

In [7]:
with open('../resources/graph_3d.pb', 'wb') as f:
    f.write(tf.get_default_graph().as_graph_def().SerializeToString())

In [8]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['input',
 'down_level_0_no_0/random_uniform/shape',
 'down_level_0_no_0/random_uniform/min',
 'down_level_0_no_0/random_uniform/max',
 'down_level_0_no_0/random_uniform/RandomUniform',
 'down_level_0_no_0/random_uniform/sub',
 'down_level_0_no_0/random_uniform/mul',
 'down_level_0_no_0/random_uniform',
 'down_level_0_no_0/kernel',
 'down_level_0_no_0/kernel/Assign',
 'down_level_0_no_0/kernel/read',
 'down_level_0_no_0/Const',
 'down_level_0_no_0/bias',
 'down_level_0_no_0/bias/Assign',
 'down_level_0_no_0/bias/read',
 'down_level_0_no_0/convolution/dilation_rate',
 'down_level_0_no_0/convolution',
 'down_level_0_no_0/Reshape/shape',
 'down_level_0_no_0/Reshape',
 'down_level_0_no_0/add',
 'batch_normalization_1/Const',
 'batch_normalization_1/gamma',
 'batch_normalization_1/gamma/Assign',
 'batch_normalization_1/gamma/read',
 'batch_normalization_1/Const_1',
 'batch_normalization_1/beta',
 'batch_normalization_1/beta/Assign',
 'batch_normalization_1/beta/read',
 'batch_normalization_

In [10]:
# save for prediction
import shutil
#shutil.rmtree("../resources/prediction")
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    tf.tables_initializer().run()
    #print(sess.graph.get_tensor_by_name('keras_learning_phase/input:0'))
    #sess.run(tf.variables_initializer([tf.compat.v1.get_variable("conv2d_1/bias:0")]))
    #sess.run(init)
    sess.run("init")
    model_input = tf.compat.v1.saved_model.build_tensor_info(sess.graph.get_tensor_by_name('input:0'))
    model_output = tf.compat.v1.saved_model.build_tensor_info(sess.graph.get_tensor_by_name('activation_11/Identity:0'))

    signature_definition = tf.compat.v1.saved_model.build_signature_def(
        inputs={"input": model_input},
        outputs={"output": model_output},
        method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME)

    builder = tf.compat.v1.saved_model.Builder("../resources/prediction_3d")
    builder.add_meta_graph_and_variables(sess, [tf.compat.v1.saved_model.tag_constants.SERVING], signature_def_map={
        tf.compat.v1.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_definition
    }, clear_devices=True)
    builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../resources/prediction_3d/saved_model.pb
